In [52]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')
import os 
import sys 
import pickle
root_path = os.path.realpath('../')
sys.path.append(root_path)

from utils.parser import parse_alldata
from utils.analysis import print_testacc, print_ttest_acc, compute_choicemats, fit_sigmoids_to_choices, gen_choicemodelrdms, stats_fit_choicerdms,fit_model_to_subjects
from utils.plotting import disp_accuracy, disp_lcurves, disp_sigmoid_fits, disp_param_estimates, disp_choicemats, disp_rsa_param_estimates, disp_model_estimates


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Parameters

In [54]:
# set a few parameters 
%matplotlib qt
working_dir = os.getcwd()
data_dir = working_dir + "/../data/animals_vehicles_1_base/"

domains = ['animals','vehicles']
curricula = ['blocked','interleaved']


# Data Import

In [55]:
# parse data 
try:
    with open(data_dir+"parsed/"+"animals_vehicles_pilot.pkl","rb") as f:
        alldata = pickle.load(f)
except FileNotFoundError:
    alldata = parse_alldata(data_dir+"raw/")
    from utils.parser import boundary_to_nan
    alldata = boundary_to_nan(alldata)
    with open(data_dir+"parsed/"+"animals_vehicles_pilot.pkl","wb") as f:
        pickle.dump(alldata,f)
    


# Analysis

## Accuracy

### calculate accuracy and test for between group differences

In [96]:

print_testacc(alldata,onlygood=True, whichtask="base")
print_ttest_acc(alldata, onlygood=True, whichtask="base")
print('\n')


** Accuracy on base task **
only good subjects (n=94): animals, blocked: 0.78
only good subjects (n=99): animals, interleaved: 0.75
only good subjects (n=96): vehicles, blocked: 0.82
only good subjects (n=99): vehicles, interleaved: 0.82
** mann whitney u on base task **
only good subjects: animals, blocked vs interleaved: p= 0.078
only good subjects: vehicles, blocked vs interleaved: p= 0.727




In [57]:
# plot test accuracy
disp_accuracy(alldata,'animals')
disp_accuracy(alldata,'vehicles')
        


In [58]:
# learning curves
disp_lcurves(alldata)



## Choice Patterns

In [66]:
choicemats = compute_choicemats(alldata)

### display sigmoids for rel and irrel dimensions 

In [93]:
betas = fit_sigmoids_to_choices(choicemats,onlygood=True,fitlapse=False)
disp_sigmoid_fits(choicemats,betas, onlygood=True)

In [97]:
# stats on estimated slopes
disp_param_estimates(betas,onlygood=True)

### display full choice matrices

In [60]:
disp_choicemats(choicemats,onlygood=True)

## Choice Pattern RSA


In [62]:
_ = gen_choicemodelrdms(monitor=True)

In [63]:
betas = stats_fit_choicerdms(choicemats,onlygood=True)
disp_rsa_param_estimates(betas,onlygood=True)

## Choice Model


In [64]:
thetas = fit_model_to_subjects(choicemats,onlygood=True)

In [98]:
# stats on estimated slopes
disp_model_estimates(thetas)